<a href="https://colab.research.google.com/github/varunpothu/Smart_HealthCare_Chatbot/blob/main/symtom2medicine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder


In [2]:
# Data Loading
file_path = "/content/drive/MyDrive/5.csv"
data = pd.read_csv(file_path)

# Display the first few rows
data.head()

<ipython-input-2-f1763da3dfd2>:3: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,...,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)


In [3]:
threshold = 0.8 * len(data)
data = data.dropna(thresh=threshold, axis=1)

In [4]:
data.head()

,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,use0,Habit Forming,Therapeutic Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,NaN,Treatment of Bacterial infections,No,ANTI INFECTIVES
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,Diarrhea,Treatment of Bacterial infections,No,ANTI INFECTIVES
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,Upset stomach,Treatment of Cough with mucus,No,RESPIRATORY
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,Nausea,Treatment of Sneezing and runny nose due to al...,No,RESPIRATORY
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,NaN,Treatment of Allergic conditions,No,RESPIRATORY


In [5]:
data = data.fillna('Unknown')


In [6]:
# Combine substitute columns
substitute_cols = [col for col in data.columns if 'substitute' in col]
data['substitutes'] = data[substitute_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
data = data.drop(columns=substitute_cols)

# Combine sideEffect columns
side_effect_cols = [col for col in data.columns if 'sideEffect' in col]
data['side_effects'] = data[side_effect_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
data = data.drop(columns=side_effect_cols)

# Combine use columns
use_cols = [col for col in data.columns if 'use' in col]
data['uses'] = data[use_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
data = data.drop(columns=use_cols)


In [7]:
data = data.drop(columns=['id', 'Habit Forming'])


In [8]:
data['uses'] = data['uses'].apply(lambda x: ', '.join(set(x.split(', '))))


In [9]:
data = data.drop_duplicates(subset=['uses'])


In [10]:
# Map uses to name
use_name_mapping = data[['uses', 'name']].drop_duplicates().set_index('uses')['name'].to_dict()

# Map name to combined substitutes and Therapeutic Class
data['substitute_therapeutic'] = data['substitutes'] + " | Therapeutic Class: " + data['Therapeutic Class']
name_mapping = data[['name', 'substitute_therapeutic']].drop_duplicates().set_index('name')['substitute_therapeutic'].to_dict()


In [11]:
import pandas as pd
import json

# Mapping uses to name
use_name_mapping = data[['uses', 'name']].drop_duplicates().set_index('uses')['name'].to_dict()

# Mapping name to combined substitutes and Therapeutic Class
data['substitute_therapeutic'] = data['substitutes'] + " | Therapeutic Class: " + data['Therapeutic Class']
name_mapping = data[['name', 'substitute_therapeutic']].drop_duplicates().set_index('name')['substitute_therapeutic'].to_dict()

# Convert mappings to DataFrames
use_name_df = pd.DataFrame(list(use_name_mapping.items()), columns=['uses', 'name'])
name_mapping_df = pd.DataFrame(list(name_mapping.items()), columns=['name', 'substitute_therapeutic'])

# Save to CSV files
use_name_df.to_csv('use_name_mapping.csv', index=False)
name_mapping_df.to_csv('name_mapping.csv', index=False)

# Save to JSON files
with open('use_name_mapping.json', 'w') as json_file:
    json.dump(use_name_mapping, json_file, indent=4)

with open('name_mapping.json', 'w') as json_file:
    json.dump(name_mapping, json_file, indent=4)

# Paths to the saved files
file_paths = {
    "Use-Name Mapping CSV": "use_name_mapping.csv",
    "Name-Substitute Therapeutic Mapping CSV": "name_mapping.csv",
    "Use-Name Mapping JSON": "use_name_mapping.json",
    "Name-Substitute Therapeutic Mapping JSON": "name_mapping.json"
}
file_paths


{'Use-Name Mapping CSV': 'use_name_mapping.csv',
 'Name-Substitute Therapeutic Mapping CSV': 'name_mapping.csv',
 'Use-Name Mapping JSON': 'use_name_mapping.json',
 'Name-Substitute Therapeutic Mapping JSON': 'name_mapping.json'}

In [12]:
import json

# Load the symptom to medication mapping
with open('use_name_mapping.json', 'r') as f:
    use_name_mapping = json.load(f)

# Load the medication to substitute and therapeutic class mapping
with open('name_mapping.json', 'r') as f:
    name_mapping = json.load(f)


from transformers import BertTokenizer, BertModel
import torch

# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Function to get BERT embedding for a given text
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()
    return embedding


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
# Generate embeddings for each symptom in the dataset
symptom_embeddings = {symptom: get_bert_embedding(symptom) for symptom in use_name_mapping.keys()}


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to find the closest matching symptom and retrieve medication
def recommend_medication(user_symptom):
    # Get the embedding for the user-provided symptom
    user_embedding = get_bert_embedding(user_symptom)

    # Calculate cosine similarity with all dataset symptoms
    similarities = {symptom: cosine_similarity([user_embedding], [embedding])[0][0] for symptom, embedding in symptom_embeddings.items()}

    # Find the closest match
    best_match = max(similarities, key=similarities.get)
    recommended_medication = use_name_mapping[best_match]

    return best_match, recommended_medication


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
file_path = 'use_name_mapping.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define the recommender class
class MedicineRecommender:
    def __init__(self, data):
        """
        Initialize the recommender with data and compute the TF-IDF matrix.
        """
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data['uses'])

    def recommend(self, user_input, top_n=3):
        """
        Recommend medicines based on the user input.

        Args:
        user_input (str): Symptoms provided by the user.
        top_n (int): Number of top recommendations to return.

        Returns:
        DataFrame: Top recommended medicines and their descriptions.
        """
        # Vectorize the user input
        user_vector = self.vectorizer.transform([user_input])
        # Compute cosine similarity
        similarity_scores = cosine_similarity(user_vector, self.tfidf_matrix)
        # Get the top N matches
        top_indices = similarity_scores.argsort()[0][-top_n:][::-1]
        recommendations = self.data.iloc[top_indices][['name', 'uses']]
        return recommendations


# Initialize the recommender
recommender = MedicineRecommender(data)

# Define a function to generate medicine recommendations for the chatbot
def get_medicine_recommendations(symptoms, top_n=3):
    """
    Get the top N medicine recommendations for the given symptoms.

    Args:
    symptoms (str): User-provided symptoms description.
    top_n (int): Number of top recommendations to return.

    Returns:
    DataFrame: Top recommended medicines and their descriptions.
    """
    return recommender.recommend(symptoms, top_n=top_n)

# Example usage: Simulating user input
example_symptoms = "cough and mucus"
recommendations = get_medicine_recommendations(example_symptoms, top_n=3)

# Display the recommendations
print("Recommendations for symptoms:", example_symptoms)
print(recommendations)


Recommendations for symptoms: cough and mucus
                  name                           uses
147   alaspan ag syrup               Cough with mucus
1     ascoril ls syrup  Treatment of Cough with mucus
53   alaspan am tablet                          Cough


In [15]:
# Example usage
user_input = "nose"
matched_symptom, recommended_medication = recommend_medication(user_input)
print(f"Matched Symptom: {matched_symptom}")
print(f"Recommended Medication: {recommended_medication}")


Matched Symptom:  Mania
Recommended Medication: a lith 450mg tablet sr
